# LangChain Application: Prompt Engineering



## Prompt and LLM

## The importance of "Prompt" in LLM

- In LangChain, **prompt templates** are a structured, reusable, and dynamic way to interact with various language models ¹. They are text strings or templates that take inputs and produce a prompt for the language model. Prompt templates offer a versatile approach to language model interactions, from setting the context and defining instructions to dynamically adjusting the content based on user needs ¹. 

- **Prompt engineering** is the process of designing and optimizing prompts to achieve the desired output from a language model ². It involves creating prompts that are tailored to specific tasks and use cases, and fine-tuning them to improve the model's performance ². Prompt engineering can include rule-based prompts, data augmentation, and prompt tuning ². 

- **Prompt design** is the process of creating prompts that are clear, concise, and effective in conveying the desired information to the language model ³. It involves crafting high-quality prompts that provide clear instructions and contextual cues to the model ³. Good prompt design is essential for obtaining accurate and relevant outputs from language models ³.



### Benefits of Prompt Templates

There are several benefits of using prompt templates in LangChain:

1. **Reusability**: Prompt templates are designed to be reusable across different language models, making it easier to create prompts for various tasks and use cases.

2. **Flexibility**: Prompt templates offer a versatile approach to language model interactions, from setting the context and defining instructions to dynamically adjusting the content based on user needs.

3. **Efficiency**: Prompt templates can save time and effort by providing a structured, reusable, and dynamic way to interact with language models.

4. **Consistency**: Prompt templates can help ensure consistency in the prompts used for different tasks and use cases, which can improve the accuracy and relevance of the model's outputs.

5. **Customizability**: Prompt templates can be customized to suit specific tasks and use cases, allowing users to fine-tune the prompts to achieve the desired output from the language model.


## Environment

- Create a python environment with the following packages installed:

```
python-dotenv
langchain
openai
youtube-transcript-api
faiss-cpu
streamlit
```

## Import

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv

import tiktoken
import os


## Environment Variables and OPENAI API
- Save personal API key in `.env` file and use `load_dotenv()` to load environment variables.
- Specify `OPENAI_API_KEY` in the self-defined `.env`.

In [2]:
load_dotenv('/Users/alvinchen/.env')

True

## Prompt Template for Proofreading

In [3]:
llm = ChatOpenAI(model='gpt-4', temperature= 0.6)

In [4]:
proofreading_instruction = """Revise the draft delimited by triple backticks\
    by following the instructions below.\
    The revision should remain within {word_limit}-word.\
    The language style should be {language_style}.\
    The genre should follow the conventions in {genre}.\
    In the revision, reduce the use of {avoid_structure}.\
    Original Draft: ```{draft}```"""

proofreading_template = ChatPromptTemplate.from_template(proofreading_instruction)

original_text = """This current study aims to investigate the prosodic properties of the speech genres (i.e., entertaining and informative) within Taiwan Mandarin YouTube content and the effectiveness of different feature modes on automatic speech genre classification models. We established a corpus consisting of 5049 utterances, each of which was an inter-pausal unit in speech. All the utterances were recorded with their corresponding transcripts, speech genres, four durational features (i.e., utterance duration, pause duration, speech rate, duration-based PVI), and three pitch-related features (i.e., f0 mean, f0 range, f0-based PVI). Based on the transcript of each utterance, we also utilized the naïve TF-IDF technique to acquire the textual features. An utterance-based analytical framework was adopted. First, we used the binary logistic model fitting on the seven proposed durational and pitch-related features to investigate which prosodic features can effectively characterize the entertaining and informative speech. Second, we built three types of automatic speech genre classification models—namely, prosody-mode, text-mode, and multimodal (prosody + text) models—to study the potentiality of prosodic features in speech genre classification and whether multimodal (prosody + text) features can further improve the performance of speech genre classification. 
According to the results of our logistic model, six of the seven proposed prosodic features—utterance duration, dur-PVI, speech rate, f0 mean, f0 range, and f0-PVI—showed statistical significance in the logistic model. Our analysis suggested that in comparison to entertaining speech, informative speech tended to have longer utterance duration, slower speech rate, more isochoric rhythm, and more perceptible intonation at a lower pitch level. Additionally, we fed the seven proposed prosodic features into the prosody-mode and multimodal speech genre classification models. Our results showed that (1) the prosody-mode model attained 0.733 accuracy, showing the potentiality of prosodic features in speech genre classification; (2) the multimodal (prosody + text) model outperformed the other single-mode ones, amounting to 0.846 accuracy. We conclude that prosodic features can complement the absent or underrepresented information of textual features in speech genre classification and further improve the already high-performing text-mode classification results. The inherent multimodality of speech makes it necessary to include both prosodic and textual features in the task of speech genre classification. 
    """

proofreading_messages = proofreading_template.format_messages(
    word_limit="50", 
    language_style="concise, simple, and easy-to-understand", 
    genre="academic journals", 
    avoid_structure = "passive voice constructions",
    draft= original_text
)

print(proofreading_messages)

[HumanMessage(content='Revise the draft delimited by triple backticks    by following the instructions below.    The revision should remain within 50-word.    The language style should be concise, simple, and easy-to-understand.    The genre should follow the conventions in academic journals.    In the revision, reduce the use of passive voice constructions.    Original Draft: ```This current study aims to investigate the prosodic properties of the speech genres (i.e., entertaining and informative) within Taiwan Mandarin YouTube content and the effectiveness of different feature modes on automatic speech genre classification models. We established a corpus consisting of 5049 utterances, each of which was an inter-pausal unit in speech. All the utterances were recorded with their corresponding transcripts, speech genres, four durational features (i.e., utterance duration, pause duration, speech rate, duration-based PVI), and three pitch-related features (i.e., f0 mean, f0 range, f0-base

In [5]:
response_revision = llm(proofreading_messages)


In [6]:
print(response_revision.content)

```The present study explores the prosodic properties of entertaining and informative speech genres in Taiwan Mandarin YouTube content to determine their impact on automatic speech genre classification. Our corpus contains 5049 utterances, each an inter-pausal speech unit, annotated with transcripts, speech genres, four durational, and three pitch-related features. We used the naïve TF-IDF technique to acquire textual features. Our binary logistic model analysis showed that six of the seven prosodic features significantly characterized entertaining and informative speech. Furthermore, our models demonstrated that prosodic features enhance speech genre classification accuracy, suggesting the necessity of including both prosodic and textual features in this task.```


## Proofreading Documents

In [26]:
import docx

doc = docx.Document('/Users/alvinchen/Dropbox/NTNUad/論文指導/Zale/thesis-final-data-all/NTNU_ENG_thesis_zale_2023.docx')

In [27]:
all_paras = doc.paragraphs
len(all_paras)

669

In [59]:
original_texts = [p.text for p in all_paras if len(p.text) > 150]
original_texts[:10]

['I would like to express my sincere gratitude to my advisor, Dr. Alvin Cheng-Hsien Chen, for his constant encouragement and strong support throughout my graduate studies. It was Dr. Chen that inspired me to probe into linguistic phenomena with a computational approach. His linguistics courses gave me a solid grounding in the basic techniques of coding. Additionally, he spent a great amount of time reading my manuscripts, always providing me with insightful and constructive suggestions in every revision. Had it not been for Dr. Chen’s assistance and guidance, this research could have never been accomplished. ',
 '\tMany thanks also go to my committee members, Dr. Li-Hsin Ning and Dr. Grace Chen-Hsiu Kuo, for their helpful comments and encouraging remarks at the proposal hearing and oral defense. They enlightened me to conduct the more comprehensive acoustic-prosodic analysis of this research. I particularly appreciate Dr. Ning’s instructions on acoustic phonetics and statistical analys

In [49]:
proofreading_messages = proofreading_template.format_messages(
    word_limit="50", 
    language_style="concise, simple, and easy-to-understand", 
    genre="academic journals", 
    avoid_structure = "passive voice constructions",
    draft= original_texts[0]
)


In [58]:
def reviseParagraph(original_text: str):
    proofreading_messages = proofreading_template.format_messages(
    word_limit="50", 
    language_style="concise, simple, and easy-to-understand", 
    genre="academic journals", 
    avoid_structure = "passive voice constructions",
    draft= original_text
)

    response_revision = llm(proofreading_messages)
    textwrap.wrap(original_text, width= 100)
    
    return response_revision.content



    

In [ ]:
import textwrap

In [62]:
out = reviseParagraph(original_text=original_texts[7])

In [63]:
textwrap.wrap(out)

['```This study examines the prosodic characteristics of entertaining',
 'and informative speech genres in Taiwan Mandarin YouTube content,',
 'utilizing a corpus of 5049 inter-pausal unit utterances. We recorded',
 'these utterances with corresponding transcripts and four durational',
 'and three pitch-related features. We used the naïve TF-IDF technique',
 'to identify textual features. Our analysis involved a binary logistic',
 'model to evaluate the effectiveness of prosodic features in',
 'characterizing speech genres and built three automatic speech genre',
 'classification models to assess the potential of prosodic features and',
 'multimodal features in enhancing genre classification performance.```']

In [61]:

textwrap.wrap(original_texts[7], width= 100)


['This current study aims to investigate the prosodic properties of the speech genres (i.e.,',
 'entertaining and informative) within Taiwan Mandarin YouTube content and the effectiveness of',
 'different feature modes on automatic speech genre classification models. We established a corpus',
 'consisting of 5049 utterances, each of which was an inter-pausal unit in speech. All the utterances',
 'were recorded with their corresponding transcripts, speech genres, four durational features (i.e.,',
 'utterance duration, pause duration, speech rate, duration-based PVI), and three pitch-related',
 'features (i.e., f0 mean, f0 range, f0-based PVI). Based on the transcript of each utterance, we also',
 'utilized the naïve TF-IDF technique to acquire the textual features. An utterance-based analytical',
 'framework was adopted. First, we used the binary logistic model fitting on the seven proposed',
 'durational and pitch-related features to investigate which prosodic features can effectively'

In [57]:

textwrap.wrap(response_revision.content, width= 100)


['```I sincerely thank my advisor, Dr. Alvin Cheng-Hsien Chen, for his consistent encouragement and',
 'robust support during my graduate studies. Dr. Chen inspired my exploration of linguistic phenomena',
 'through a computational approach. His linguistics courses equipped me with fundamental coding',
 'techniques. He invested substantial time in reviewing my manuscripts, offering insightful and',
 "constructive suggestions for each revision. Without Dr. Chen's guidance, this research would not",
 'have been possible.```']

## Resources

- [OPENAI API Models](https://platform.openai.com/docs/models): For more detail information about the models supported by OPENAI.